In [27]:
import faiss
import hnswlib
import numpy as np

np.random.seed(1234)

In [28]:
def sample_vector(distribution, shape, **kwargs):
    # shape = (num_element, dim)
    # Example usage:
    # vec1 = sample_vector('normal', (5,), mean=0, std=1)
    # vec2 = sample_vector('uniform', (5,), low=0, high=1)
    if distribution == 'normal':
        return np.random.normal(kwargs.get('mean', 0), kwargs.get('std', 1), size=shape).astype('float32')
    elif distribution == 'uniform':
        return np.random.uniform(kwargs.get('low', 0), kwargs.get('high', 1), size=shape).astype('float32')
    else:
        return "Unsupported distribution"

def find_KNN(dataset, query_vector, k):
    # find kNN of the given query in the dataset. query_vector should be one vector whith shape (1, ndim) or a ndim array
    
    # Calculate distances from query_vector to all vectors in dataset
    distances = np.linalg.norm(np.array(dataset) - np.array(query_vector).flatten(), axis=1)

    # Get the indices of the k smallest distances
    k_indices = np.argsort(distances)[:k]
    
    # Get k nearest vectors and their distances
    k_nearest_vectors = [dataset[i] for i in k_indices]
    k_nearest_distances = [distances[i] for i in k_indices]
    
    return k_indices, k_nearest_vectors, k_nearest_distances

def topk_vector_accuracy(topk_vectors, retrieved_vectors):
    # return accuracy score numbers of match vector / k
    # Count matching vectors
    count_matches = 0
    for gt, est in zip(topk_vectors, retrieved_vectors):
        match = gt==est
        if match.all(): count_matches+=1 
    
    # Calculate accuracy
    k = len(topk_vectors)
    accuracy = count_matches / k
    return accuracy

def topk_idx_accuracy(topk_idx, retrieved_idx):
    # return accuracy score numbers of match vector / k
    pro_topk_idx = np.int64(topk_idx.flatten())
    pro_retrieved_idx = np.int64(retrieved_idx.flatten())
    
    # Count matching vectors
    matches = pro_topk_idx==pro_retrieved_idx
    num_matches = sum(matches.flatten())
    # print(num_matches)
    
    # Calculate accuracy
    k = len(pro_topk_idx)
    accuracy = num_matches / k
    return accuracy

In [29]:
dim = 32
num_elements = 10000
k = 10
nlist = 100

# Generating sample data
dataset = np.float32(sample_vector('normal', (num_elements, dim), mean=0, std=1))

quantizer = faiss.IndexFlatL2(dim)  # the other index
index = faiss.IndexIVFFlat(quantizer, dim, nlist)
index.train(dataset)
print(index.is_trained)
index.add(dataset)                  # add vectors to the index
print(index.ntotal)

True
10000


In [30]:
num_runs = 1000
accs = []
for i in range(num_runs):
    vec1 = sample_vector('normal', (1, dim), mean=0, std=1)
    distances, est_idx = index.search(vec1, k=k)
    gt_idx, gt_vecs, gt_distances = find_KNN(dataset, vec1, k=k)
    acc = topk_idx_accuracy(est_idx, gt_idx)
    accs.append(acc)

print("{} runs; top_{} Acc:".format(num_runs, k), np.average(accs))

1000 runs; top_10 Acc: 0.018600000000000002


In [31]:
new_data_size = num_elements
new_mean = 0.1
new_std = 1.1

newDataset = sample_vector('normal', (new_data_size, dim), mean=new_mean, std=new_std)
dataset = np.concatenate([dataset, newDataset])

print(index.is_trained)
index.add(newDataset)                  # add vectors to the index

print("current DB size:", index.ntotal)

True
current DB size: 20000


In [32]:
num_runs = 1000
accs = []
for i in range(num_runs):
    vec1 = sample_vector('normal', (1, dim), mean=new_mean, std=new_std)
    distances, est_idx = index.search(vec1, k=k)
    gt_idx, gt_vecs, gt_distances = find_KNN(dataset, vec1, k=k)
    acc = topk_idx_accuracy(est_idx, gt_idx)
    accs.append(acc)

print("{} runs; top_{} Acc:".format(num_runs, k), np.average(accs))

1000 runs; top_10 Acc: 0.023


In [33]:
index.train(newDataset)

In [34]:
num_runs = 1000
accs = []
for i in range(num_runs):
    vec1 = sample_vector('normal', (1, dim), mean=new_mean, std=new_std)
    distances, est_idx = index.search(vec1, k=k)
    gt_idx, gt_vecs, gt_distances = find_KNN(dataset, vec1, k=k)
    acc = topk_idx_accuracy(est_idx, gt_idx)
    accs.append(acc)

print("{} runs; top_{} Acc:".format(num_runs, k), np.average(accs))

1000 runs; top_10 Acc: 0.02


In [22]:
index.this

<Swig Object of type 'faiss::IndexFlatL2 *' at 0x11e518cc0>